In [86]:
import ccxt
import pandas as pd

# Initialize Binance exchange
exchange = ccxt.binance()

# Fetch 1-minute OHLCV data
symbol = "BTC/USDT"
timeframe = "1m"  # Fetch 1-minute data
limit = 1000000  # Fetch last 1000 minutes (~16.6 hours)

ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

# Convert to DataFrame
df = pd.DataFrame(ohlcv, columns=["Timestamp", "Open", "High", "Low", "Close", "Volume"])
df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit="ms")
df.set_index("Timestamp", inplace=True)

In [87]:
# Resample to 10-minute, 30-minute, and 4-hour data
df_10min = df.resample("10T").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume": "sum"
}).dropna()

df_30min = df.resample("30T").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume": "sum"
}).dropna()

df_4hour = df.resample("4H").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume": "sum"
}).dropna()


/var/folders/g9/01xw9sf55fbg7wx6rnn2dsc40000gn/T/ipykernel_3583/2634678719.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_10min = df.resample("10T").agg({
/var/folders/g9/01xw9sf55fbg7wx6rnn2dsc40000gn/T/ipykernel_3583/2634678719.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_30min = df.resample("30T").agg({
/var/folders/g9/01xw9sf55fbg7wx6rnn2dsc40000gn/T/ipykernel_3583/2634678719.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4hour = df.resample("4H").agg({


In [88]:
# Fractal identification function
def identify_fractals(df):
    df["Bullish_Fractal"] = (df["Low"].shift(2) > df["Low"].shift(1)) & (df["Low"].shift(2) > df["Low"]) & \
                            (df["Low"].shift(2) > df["Low"].shift(-1)) & (df["Low"].shift(2) > df["Low"].shift(-2))

    df["Bearish_Fractal"] = (df["High"].shift(2) < df["High"].shift(1)) & (df["High"].shift(2) < df["High"]) & \
                            (df["High"].shift(2) < df["High"].shift(-1)) & (df["High"].shift(2) < df["High"].shift(-2))

    return df

# Apply fractal detection to 10-minute, 30-minute, and 4-hour data
df_10min = identify_fractals(df_10min)
df_30min = identify_fractals(df_30min)
df_4hour = identify_fractals(df_4hour)

# Check for matching patterns between 10-minute and 30-minute
df_10min['Pattern_Match_30min'] = df_10min['Bullish_Fractal'] & df_30min['Bullish_Fractal'].reindex(df_10min.index, method='ffill') | \
                                  df_10min['Bearish_Fractal'] & df_30min['Bearish_Fractal'].reindex(df_10min.index, method='ffill')

# Resample to 4-hour data and check if pattern matches
df_10min_30min_match = df_10min[df_10min['Pattern_Match_30min']].copy()

# Reset the index to merge with 4-hour data
df_10min_30min_match = df_10min_30min_match.reset_index()
df_4hour = df_4hour.reset_index()

# Rename columns for clarity to avoid column conflicts
df_10min_30min_match = df_10min_30min_match.rename(columns={
    'Timestamp': 'Fractal_Timestamp',
    'Open': 'df10min_Open', 
    'High': 'df10min_High', 
    'Low': 'df10min_Low', 
    'Close': 'df10min_Close', 
    'Volume': 'df10min_Volume',
    'Bullish_Fractal': 'df10min_Bullish_Fractal',
    'Bearish_Fractal': 'df10min_Bearish_Fractal'
})

df_4hour = df_4hour.rename(columns={
    'Timestamp': 'Fractal_Timestamp',
    'Open': 'df4hour_Open', 
    'High': 'df4hour_High', 
    'Low': 'df4hour_Low', 
    'Close': 'df4hour_Close', 
    'Volume': 'df4hour_Volume',
    'Bullish_Fractal': 'df4hour_Bullish_Fractal',
    'Bearish_Fractal': 'df4hour_Bearish_Fractal'
})

In [89]:
# Merge the data based on Fractal_Timestamp
df_trade_decision = pd.merge_asof(
    df_10min_30min_match.sort_values("Fractal_Timestamp"),
    df_4hour.sort_values("Fractal_Timestamp"),
    on="Fractal_Timestamp",
    direction="backward"
)

# Drop rows where no match was found
df_trade_decision = df_trade_decision.dropna()

# Display the matched patterns with 4-hour trade decision
print("\nTrade Decision Based on Matched Patterns:\n", df_trade_decision)

# Logic for making a trade (example)
# If we find a matching pattern, you might decide to trade at the open of the next 4-hour bar.
# Example decision rule: Buy if the next 4-hour close is higher than the current close
df_trade_decision['Trade_Decision'] = df_trade_decision['df4hour_Close'] > df_trade_decision['df10min_Close']

# Display the trade decision
print("\nTrade Decision (Buy if True, else No Trade):\n", df_trade_decision[['Fractal_Timestamp', 'Trade_Decision']])


Trade Decision Based on Matched Patterns:
      Fractal_Timestamp  df10min_Open  df10min_High  df10min_Low  \
0  2025-01-29 16:30:00     101877.99     102108.00    101814.18   
1  2025-01-29 16:40:00     101979.71     102150.00    101964.00   
2  2025-01-29 16:50:00     102038.01     102136.80    101949.02   
3  2025-01-29 17:10:00     102192.66     102444.00    102102.69   
4  2025-01-29 17:20:00     102367.27     102415.20    102260.84   
5  2025-01-29 17:40:00     102649.67     102686.05    102500.00   
6  2025-01-29 20:30:00     104365.99     104782.68    104066.66   
7  2025-01-29 21:50:00     104037.03     104087.42    103715.50   
8  2025-01-29 22:00:00     103741.11     103831.56    103475.00   
9  2025-01-29 22:10:00     103475.00     103589.00    103405.77   
10 2025-01-30 01:00:00     104026.05     104119.24    103925.00   
11 2025-01-30 01:20:00     104311.51     104499.00    104136.01   
12 2025-01-30 01:30:00     104372.23     104555.00    104148.79   
13 2025-01-30 03:0

In [90]:
# Add a column for the next 4-hour close price
df_trade_decision['Next_4H_Close'] = df_trade_decision['df4hour_Close'].shift(-1)

# Evaluate if the trade decision was good or bad
df_trade_decision['Trade_Result'] = df_trade_decision.apply(
    lambda row: "Good" if (row['Trade_Decision'] and row['Next_4H_Close'] > row['df4hour_Close'])
    else ("Bad" if row['Trade_Decision'] else "No Trade"), axis=1
)


In [91]:
# Add a column for the next 4-hour open price (Trade Entry Price)
df_trade_decision['Trade_Price'] = df_trade_decision['df4hour_Open'].shift(-1)

# If Trade_Decision is False, set Trade_Price to NaN
df_trade_decision.loc[~df_trade_decision['Trade_Decision'], 'Trade_Price'] = None

# Display the trade results with the trade price
print("\nTrade Decisions with Trade Price:\n", df_trade_decision[['Fractal_Timestamp', 'Trade_Decision', 'Trade_Price']])



Trade Decisions with Trade Price:
      Fractal_Timestamp  Trade_Decision  Trade_Price
0  2025-01-29 16:30:00            True    101980.78
1  2025-01-29 16:40:00            True    101980.78
2  2025-01-29 16:50:00            True    101980.78
3  2025-01-29 17:10:00            True    101980.78
4  2025-01-29 17:20:00            True    101980.78
5  2025-01-29 17:40:00            True    103584.00
6  2025-01-29 20:30:00           False          NaN
7  2025-01-29 21:50:00           False          NaN
8  2025-01-29 22:00:00            True    103584.00
9  2025-01-29 22:10:00            True    103733.25
10 2025-01-30 01:00:00            True    103733.25
11 2025-01-30 01:20:00            True    103733.25
12 2025-01-30 01:30:00            True    103733.25
13 2025-01-30 03:00:00            True    103733.25
14 2025-01-30 03:10:00            True    103733.25
15 2025-01-30 03:40:00            True    103733.25
16 2025-01-30 03:50:00           False          NaN
17 2025-01-30 05:00:00      

In [92]:
# Add a column for trade exit price (Close of the next 4-hour candle)
df_trade_decision['Exit_Price'] = df_trade_decision['df4hour_Close'].shift(-1)

# If Trade_Decision is False, set Exit_Price to NaN
df_trade_decision.loc[~df_trade_decision['Trade_Decision'], 'Exit_Price'] = None

# Calculate Profit/Loss (Exit Price - Trade Price)
df_trade_decision['Profit_Loss'] = df_trade_decision['Exit_Price'] - df_trade_decision['Trade_Price']

# If Trade_Decision is False, set Profit_Loss to NaN
df_trade_decision.loc[~df_trade_decision['Trade_Decision'], 'Profit_Loss'] = None

# Display trade decisions with Profit/Loss
print("\nTrade Results with Profit/Loss:\n", df_trade_decision[['Fractal_Timestamp', 'Trade_Decision', 'Trade_Price', 'Exit_Price', 'Profit_Loss']])



Trade Results with Profit/Loss:
      Fractal_Timestamp  Trade_Decision  Trade_Price  Exit_Price  Profit_Loss
0  2025-01-29 16:30:00            True    101980.78   103583.99      1603.21
1  2025-01-29 16:40:00            True    101980.78   103583.99      1603.21
2  2025-01-29 16:50:00            True    101980.78   103583.99      1603.21
3  2025-01-29 17:10:00            True    101980.78   103583.99      1603.21
4  2025-01-29 17:20:00            True    101980.78   103583.99      1603.21
5  2025-01-29 17:40:00            True    103584.00   103733.24       149.24
6  2025-01-29 20:30:00           False          NaN         NaN          NaN
7  2025-01-29 21:50:00           False          NaN         NaN          NaN
8  2025-01-29 22:00:00            True    103584.00   103733.24       149.24
9  2025-01-29 22:10:00            True    103733.25   105196.00      1462.75
10 2025-01-30 01:00:00            True    103733.25   105196.00      1462.75
11 2025-01-30 01:20:00            True    

In [94]:
# Holding period (Assuming we hold for one 4-hour candle)
df_trade_decision['Holding_Period_Hours'] = 4

# Return percentage calculation
df_trade_decision['Return_%'] = (df_trade_decision['Profit_Loss'] / df_trade_decision['Trade_Price']) * 100

# Track cumulative profit/loss
df_trade_decision['Cumulative_P&L'] = df_trade_decision['Profit_Loss'].cumsum()

# Count wins and losses
df_trade_decision['Win'] = df_trade_decision['Profit_Loss'] > 0
df_trade_decision['Loss'] = df_trade_decision['Profit_Loss'] < 0

# Cumulative win/loss count
df_trade_decision['Win_Count'] = df_trade_decision['Win'].cumsum()
df_trade_decision['Loss_Count'] = df_trade_decision['Loss'].cumsum()

# Determine trade direction (Long or Short)
df_trade_decision['Trade_Type'] = df_trade_decision.apply(
    lambda row: 'Long' if row['df10min_Bullish_Fractal'] else 'Short' if row['df10min_Bearish_Fractal'] else 'No Trade', 
    axis=1
)

In [107]:
# Get the next two 4-hour candles after the trade
df_trade_decision['Next_4H_Open_1'] = df_trade_decision['df4hour_Open'].shift(-1)
df_trade_decision['Next_4H_Close_1'] = df_trade_decision['df4hour_Close'].shift(-1)

df_trade_decision['Next_4H_Open_2'] = df_trade_decision['df4hour_Open'].shift(-2)
df_trade_decision['Next_4H_Close_2'] = df_trade_decision['df4hour_Close'].shift(-2)

# Determine if each candle is bullish or bearish
df_trade_decision['Next_4H_Trend_1'] = df_trade_decision.apply(
    lambda row: "Bullish" if row['Next_4H_Close_1'] > row['Next_4H_Open_1'] else "Bearish", axis=1
)

df_trade_decision['Next_4H_Trend_2'] = df_trade_decision.apply(
    lambda row: "Bullish" if row['Next_4H_Close_2'] > row['Next_4H_Open_2'] else "Bearish", axis=1
)

# Determine the overall trend after trade
def determine_trend(row):
    if row['Next_4H_Trend_1'] == "Bullish" and row['Next_4H_Trend_2'] == "Bullish":
        return "Uptrend"
    elif row['Next_4H_Trend_1'] == "Bearish" and row['Next_4H_Trend_2'] == "Bearish":
        return "Downtrend"
    else:
        return "Mixed/Uncertain"

df_trade_decision['Post_Trade_Trend'] = df_trade_decision.apply(determine_trend, axis=1)

# Display trade decisions with post-trade trend
print("\nTrade Results with Post-Trade Trend:\n") 
df_final=df_trade_decision[['Fractal_Timestamp','df10min_Bullish_Fractal','df10min_Bearish_Fractal','Pattern_Match_30min',
                   'Trade_Decision', 'Trade_Price', 'Exit_Price', 
                         'Next_4H_Trend_1', 'Next_4H_Trend_2', 'Post_Trade_Trend',
                  'Trade_Type', 'Profit_Loss', 'Return_%', 'Holding_Period_Hours', 
                'Cumulative_P&L', 'Win_Count', 'Loss_Count']]



Trade Results with Post-Trade Trend:



In [109]:
df_final

,Fractal_Timestamp,df10min_Bullish_Fractal,df10min_Bearish_Fractal,Pattern_Match_30min,Trade_Decision,Trade_Price,Exit_Price,Next_4H_Trend_1,Next_4H_Trend_2,Post_Trade_Trend,Trade_Type,Profit_Loss,Return_%,Holding_Period_Hours,Cumulative_P&L,Win_Count,Loss_Count
0,2025-01-29 16:30:00,False,True,True,True,101980.78,103583.99,Bullish,Bullish,Uptrend,Short,1603.21,1.572071,4,1603.21,1,0
1,2025-01-29 16:40:00,False,True,True,True,101980.78,103583.99,Bullish,Bullish,Uptrend,Short,1603.21,1.572071,4,3206.42,2,0
2,2025-01-29 16:50:00,False,True,True,True,101980.78,103583.99,Bullish,Bullish,Uptrend,Short,1603.21,1.572071,4,4809.63,3,0
3,2025-01-29 17:10:00,False,True,True,True,101980.78,103583.99,Bullish,Bullish,Uptrend,Short,1603.21,1.572071,4,6412.84,4,0
4,2025-01-29 17:20:00,False,True,True,True,101980.78,103583.99,Bullish,Bullish,Uptrend,Short,1603.21,1.572071,4,8016.05,5,0
5,2025-01-29 17:40:00,False,True,True,True,103584.00,103733.24,Bullish,Bullish,Uptrend,Short,149.24,0.144076,4,8165.29,6,0
6,2025-01-29 20:30:00,False,True,True,False,NaN,NaN,Bullish,Bullish,Uptrend,Short,NaN,NaN,4,NaN,6,0
7,2025-01-29 21:50:00,True,False,True,False,NaN,NaN,Bullish,Bullish,Uptrend,Long,NaN,NaN,4,NaN,6,0
8,2025-01-29 22:00:00,True,False,True,True,103584.00,103733.24,Bullish,Bullish,Uptrend,Long,149.24,0.144076,4,8314.53,7,0
9,2025-01-29 22:10:00,True,False,True,True,103733.25,105196.00,Bullish,Bullish,Uptrend,Long,1462.75,1.410107,4,9777.28,8,0
